In [ ]:
!pip install transformers datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f83b269d4776bc1fa64fb7984eaf5e6a1f0532a2a8b2ad3977820e8e90dd91e4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset('ccdv/pubmed-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
data=dataset['test']

In [ ]:
def remove_newlines(example):
    example['article'] = example['article'].replace('\n', '')
    return example
data = data.map(remove_newlines)

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

In [ ]:
data = data.select(range(500))

In [ ]:
from datasets import load_metric
import torch
rouge = load_metric("rouge")

<ipython-input-16-10357efb0a5b>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "sshleifer/distilbart-cnn-12-6"
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def generate_summary(batch):
    inputs = tokenizer(batch["article"], return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    batch["pred"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return batch

In [ ]:
result = data.map(generate_summary)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
rouge_scores = rouge.compute(predictions=result["pred"], references=data["abstract"])
print(rouge_scores)

{'rouge1': AggregateScore(low=Score(precision=0.4658745448219647, recall=0.16864903007263443, fmeasure=0.23584676364639884), mid=Score(precision=0.4777721949846605, recall=0.17579128979309122, fmeasure=0.24380288595922495), high=Score(precision=0.4914670049226041, recall=0.18331362668370077, fmeasure=0.25147943966841096)), 'rouge2': AggregateScore(low=Score(precision=0.1521526548566634, recall=0.053582515604703715, fmeasure=0.07579185199301924), mid=Score(precision=0.1646255052351207, recall=0.058547979019289556, fmeasure=0.08204844649982637), high=Score(precision=0.1801347834253245, recall=0.06407746006999293, fmeasure=0.08924313019743305)), 'rougeL': AggregateScore(low=Score(precision=0.30010950232486167, recall=0.10872223637324648, fmeasure=0.15225170014929393), mid=Score(precision=0.3128649017805992, recall=0.11433679762519275, fmeasure=0.15868543756660958), high=Score(precision=0.32622984163647095, recall=0.12014495077626089, fmeasure=0.1649756546083066)), 'rougeLsum': AggregateSc

In [ ]:
from typing import Dict, Any

def simplify_rouge_scores(rouge_scores: Dict[str, Any]) -> str:
    simplified_text = ""
    for key, value in rouge_scores.items():
        # Extract low, mid, and high scores for each ROUGE metric
        low, mid, high = value.low, value.mid, value.high
        simplified_text += f"{key}: Precision ranges from {low.precision:.2%} to {high.precision:.2%}, "
        simplified_text += f"Recall ranges from {low.recall:.2%} to {high.recall:.2%}, "
        simplified_text += f"F1 Score ranges from {low.fmeasure:.2%} to {high.fmeasure:.2%}.\n"

    return simplified_text

In [ ]:
print(simplify_rouge_scores(rouge_scores))

rouge1: Precision ranges from 46.59% to 49.15%, Recall ranges from 16.86% to 18.33%, F1 Score ranges from 23.58% to 25.15%.
rouge2: Precision ranges from 15.22% to 18.01%, Recall ranges from 5.36% to 6.41%, F1 Score ranges from 7.58% to 8.92%.
rougeL: Precision ranges from 30.01% to 32.62%, Recall ranges from 10.87% to 12.01%, F1 Score ranges from 15.23% to 16.50%.
rougeLsum: Precision ranges from 39.40% to 42.10%, Recall ranges from 14.22% to 15.49%, F1 Score ranges from 19.94% to 21.34%.

